<a href="https://colab.research.google.com/github/tiaaburton/Milestone-II-Machine-Learning-Project/blob/main/Playground_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install pyspark

In [ ]:
! pip install pyspark[pandas-on-spark] plotly

In [ ]:
! pip install chart_studio

     |████████████████████████████████| 64 kB 1.7 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=a51b7f84eba928f1bdb6b72df6f72fa1dd912868e442cb1e73e3546b14e57ee8
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [ ]:
import pandas as pd
import numpy as np
import re
import os
import sys

from google.cloud import bigquery
import pyarrow
from datetime import datetime
from tqdm import tqdm

from oauth2client.client import GoogleCredentials
from google.colab import auth, files

import ast
import json
import re
import random

import pyspark
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
import pyspark.ml
from pyspark.ml import *



In [ ]:
RANDOM_SEED = 655

In [ ]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Binary Buyer Prediction") \
    .getOrCreate()

In [ ]:
sample = spark.read.csv('/content/drive/Shareddrives/SIADS - 694-695 Team Drive/processed_file.csv', header=True).withColumnRenamed('_c0','index')

In [ ]:
from collections import Counter
core = [col for col in sample.columns if '.' not in col]
additional = [col for col in sample.columns if '.' in col]
second = []
third = []

for col in additional:
  count = Counter(col)
  if count['.'] == 1:
    second.append(col)
  elif count['.'] == 2:
    third.append(col)

In [ ]:
# columns = []
# with open('/content/drive/Shareddrives/SIADS - 694-695 Team Drive/Notebooks/final_columns.txt') as f:
#   for ln in f.readlines():
#     saved = ln.strip().split('\'')
#     columns.append(saved[1])
# print(columns)

In [ ]:
processed = pd.read_csv('/content/drive/Shareddrives/SIADS - 694-695 Team Drive/processed_file.csv',index_col=0)

In [ ]:
processed.columns

Index(['socialEngagementType', 'hits', 'channelGrouping', 'trafficSource',
       'totals', 'date', 'device', 'fullVisitorId', 'geoNetwork',
       'totals_visits', 'totals_hits', 'totals_pageviews', 'totals_timeOnSite',
       'totals_transactions', 'totals_newVisits', 'totals_screenviews',
       'totals_uniqueScreenviews', 'totals_timeOnScreen',
       'totals_totalTransactionRevenue', 'hits_type', 'geoNetwork_country',
       'trafficSource_source', 'trafficSource_medium',
       'trafficSource_isTrueDirect', 'device_browser',
       'device_operatingSystem', 'device_deviceCategory', 'hits_hour',
       'hits_minute', 'hits_transaction', 'hits_item', 'hits_dataSource',
       'hits_appInfo', 'hits_eCommerceAction', 'totals_bounces',
       'hits_appInfo_screenDepth', 'hits_eCommerceAction_action_type',
       'hits_transaction_transactionRevenue', 'hits_item_productName',
       'hits_item_productCategory', 'hits_item_itemRevenue'],
      dtype='object')

In [ ]:
processed.describe()

,date,fullVisitorId,totals.visits,totals.screenviews,totals.totalTransactionRevenue,totals.bounces,totals.uniqueScreenviews,totals.newVisits,totals.hits,totals.transactions,totals.timeOnScreen,totals.timeOnSite,totals.pageviews
count,3.572500e+04,3.572500e+04,35725.0,0.0,3.630000e+02,18244.0,0.0,28977.0,35725.000000,369.000000,0.0,17447.000000,35718.000000
mean,2.016099e+07,4.537203e+18,1.0,NaN,1.565480e+08,1.0,NaN,1.0,4.740770,1.054201,NaN,242.206626,3.883980
std,1.128160e+02,3.077969e+18,0.0,NaN,2.527672e+08,0.0,NaN,0.0,11.530412,0.308027,NaN,483.392041,8.217082
min,2.016080e+07,3.347106e+13,1.0,NaN,3.560000e+06,1.0,NaN,1.0,1.000000,1.000000,NaN,1.000000,1.000000
25%,2.016091e+07,1.624230e+18,1.0,NaN,3.173500e+07,1.0,NaN,1.0,1.000000,1.000000,NaN,33.000000,1.000000
50%,2.016101e+07,4.444965e+18,1.0,NaN,6.399000e+07,1.0,NaN,1.0,1.000000,1.000000,NaN,72.000000,1.000000
75%,2.016111e+07,7.227935e+18,1.0,NaN,1.496200e+08,1.0,NaN,1.0,4.000000,1.000000,NaN,219.000000,3.000000
max,2.016113e+07,9.999397e+18,1.0,NaN,2.213380e+09,1.0,NaN,1.0,500.000000,4.000000,NaN,11094.000000,469.000000


In [ ]:
processed = processed.drop(['totals.screenviews','totals.uniqueScreenviews','totals.timeOnScreen',
                            'device','totals','hits','hits.type','trafficSource','geoNetwork'],axis=1)

In [ ]:
for c in processed.columns:
  if '.' in c:
    repl = c.replace('.','_')
    processed = processed.rename(columns={c: repl},inplace=True) 

In [ ]:
processed.head()

,socialEngagementType,hits,channelGrouping,trafficSource,totals,date,device,fullVisitorId,geoNetwork,totals_visits,...,hits_dataSource,hits_appInfo,hits_eCommerceAction,totals_bounces,hits_appInfo_screenDepth,hits_eCommerceAction_action_type,hits_transaction_transactionRevenue,hits_item_productName,hits_item_productCategory,hits_item_itemRevenue
0,Not Socially Engaged,"[{'hitNumber': 1, 'time': 0, 'hour': 6, 'minut...",Social,"{'referralPath': '/', 'campaign': '(not set)',...","{'visits': 1, 'hits': 10, 'pageviews': 9, 'tim...",20160801,"{'browser': 'Chrome', 'browserVersion': 'not a...",5160807529212499245,"{'continent': 'Europe', 'subContinent': 'Easte...",1,...,"[None, None, None, None, None, None, None, Non...","[{'name': None, 'version': None, 'id': None, '...","[{'action_type': '0', 'step': 1, 'option': Non...",NaN,0,0,NaN,NaN,NaN,NaN
1,Not Socially Engaged,"[{'hitNumber': 1, 'time': 0, 'hour': 22, 'minu...",Organic Search,"{'referralPath': None, 'campaign': '(not set)'...","{'visits': 1, 'hits': 2, 'pageviews': 2, 'time...",20160801,"{'browser': 'Chrome', 'browserVersion': 'not a...",558025075151655343,"{'continent': 'Americas', 'subContinent': 'Nor...",1,...,"[None, None]","[{'name': None, 'version': None, 'id': None, '...","[{'action_type': '0', 'step': 1, 'option': Non...",NaN,0,0,NaN,NaN,NaN,NaN
2,Not Socially Engaged,"[{'hitNumber': 1, 'time': 0, 'hour': 13, 'minu...",Direct,"{'referralPath': None, 'campaign': '(not set)'...","{'visits': 1, 'hits': 2, 'pageviews': 2, 'time...",20160801,"{'browser': 'Chrome', 'browserVersion': 'not a...",3751823672052015547,"{'continent': 'Americas', 'subContinent': 'Nor...",1,...,"[None, None]","[{'name': None, 'version': None, 'id': None, '...","[{'action_type': '0', 'step': 1, 'option': Non...",NaN,0,0,NaN,NaN,NaN,NaN
3,Not Socially Engaged,"[{'hitNumber': 1, 'time': 0, 'hour': 10, 'minu...",Direct,"{'referralPath': None, 'campaign': '(not set)'...","{'visits': 1, 'hits': 3, 'pageviews': 3, 'time...",20160801,"{'browser': 'Chrome', 'browserVersion': 'not a...",5231073281895209426,"{'continent': 'Americas', 'subContinent': 'Nor...",1,...,"[None, None, None]","[{'name': None, 'version': None, 'id': None, '...","[{'action_type': '0', 'step': 1, 'option': Non...",NaN,0,0,NaN,NaN,NaN,NaN
4,Not Socially Engaged,"[{'hitNumber': 1, 'time': 0, 'hour': 8, 'minut...",Display,"{'referralPath': None, 'campaign': '(not set)'...","{'visits': 1, 'hits': 3, 'pageviews': 3, 'time...",20160801,"{'browser': 'Chrome', 'browserVersion': 'not a...",7157626079567895669,"{'continent': 'Americas', 'subContinent': 'Nor...",1,...,"[None, None, None]","[{'name': None, 'version': None, 'id': None, '...","[{'action_type': '0', 'step': 1, 'option': Non...",NaN,0,0,NaN,NaN,NaN,NaN


In [ ]:
train = processed[processed.columns[~processed.columns.isin(['totals_transactions'])]]
train = train[['date','fullVisitorId','totals_visits','totals_totalTransactionRevenue',
               'totals_bounces','totals_newVisits','totals_hits','totals_timeOnSite',
               'totals_pageviews']]
               
target = processed[processed.columns[processed.columns.isin(['totals_transactions'])]]
target = pd.Categorical(target.iloc[:,0])
target = pd.Series(target.add_categories(0.0).fillna(0.0))

In [ ]:
train.corr()

,date,fullVisitorId,totals_visits,totals_totalTransactionRevenue,totals_bounces,totals_newVisits,totals_hits,totals_timeOnSite,totals_pageviews
date,1.000000,-0.006308,NaN,0.047847,NaN,NaN,-0.001473,0.010124,0.006483
fullVisitorId,-0.006308,1.000000,NaN,0.060864,NaN,NaN,0.003682,-0.006080,0.003720
totals_visits,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
totals_totalTransactionRevenue,0.047847,0.060864,NaN,1.000000,NaN,NaN,0.185626,0.135123,0.184333
totals_bounces,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
totals_newVisits,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
totals_hits,-0.001473,0.003682,NaN,0.185626,NaN,NaN,1.000000,0.623907,0.987265
totals_timeOnSite,0.010124,-0.006080,NaN,0.135123,NaN,NaN,0.623907,1.000000,0.644480
totals_pageviews,0.006483,0.003720,NaN,0.184333,NaN,NaN,0.987265,0.644480,1.000000


In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import recall_score

import pyspark.pandas as ps
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.feature import *
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql import SQLContext

import chart_studio.plotly as py
import plotly.graph_objs as go
import requests
requests.packages.urllib3.disable_warnings()

In [ ]:
spark.newSession()

In [ ]:
train = train.fillna(0)
train.dtypes, target.dtype

(date                                int64
 fullVisitorId                      uint64
 totals_visits                       int64
 totals_totalTransactionRevenue    float64
 totals_bounces                    float64
 totals_newVisits                  float64
 totals_hits                         int64
 totals_timeOnSite                 float64
 totals_pageviews                  float64
 dtype: object,
 CategoricalDtype(categories=[1.0, 2.0, 3.0, 5.0, 0.0], ordered=False))

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=.3, stratify=target)

In [ ]:
dum_clf = DummyClassifier(strategy='stratified', random_state=RANDOM_SEED)
# lr = LogisticRegression(solver='lbfgs', max_iter=40000)
dtree = DecisionTreeClassifier(criterion='entropy',random_state=RANDOM_SEED)

In [ ]:
train_spark = processed[['date','fullVisitorId','totals_visits','totals_totalTransactionRevenue',
               'totals_bounces','totals_newVisits','totals_hits','totals_timeOnSite',
               'totals_pageviews', 'totals_transactions']]

train_spark = spark.createDataFrame(train_spark).withColumn('label', col('totals_transactions'))
train_spark = (train_spark.withColumn('label', when(train_spark['totals_transactions'] == np.nan, 0.0).otherwise(1.0)))
train_spark = train_spark.dropna(subset=['fullVisitorId'])

# train_spark.select('*').filter(col('label')==0.0).head(2)
train_spark.select([(count(when(col(c).contains('None') | \
                              col(c).contains('NULL') | \
                              (col(c) == '') | \
                              col(c).isNull() | \
                              isnan(c) , c)) / count('*')).alias(c) for c in train_spark.columns]).show()


+----+-------------+-------------+------------------------------+------------------+------------------+-----------+------------------+--------------------+-------------------+-----+
|date|fullVisitorId|totals_visits|totals_totalTransactionRevenue|    totals_bounces|  totals_newVisits|totals_hits| totals_timeOnSite|    totals_pageviews|totals_transactions|label|
+----+-------------+-------------+------------------------------+------------------+------------------+-----------+------------------+--------------------+-------------------+-----+
| 0.0|          0.0|          0.0|            0.9883119946814214|0.4987025241802312|0.2068669711981814|        0.0|0.5023697698856935|8.578352527397113E-5| 0.9882691029187844|  0.0|
+----+-------------+-------------+------------------------------+------------------+------------------+-----------+------------------+--------------------+-------------------+-----+



In [ ]:
# X_train_spark = spark.createDataFrame(X_train)
# X_val_spark = spark.createDataFrame(X_val)
# y_train_spark = ps.from_pandas(y_train)
# y_val_spark = ps.from_pandas(y_val)

In [ ]:
for c in train_spark.columns:
  if c != 'fullVisitorId':
    train_spark = train_spark.withColumn(c, 
                                  train_spark[c]
                                  .cast('int'))
train_spark.head()

Row(date=20160801, fullVisitorId=5160807529212499245, totals_visits=1, totals_bounces=0, totals_newVisits=1, totals_hits=10, totals_timeOnSite=148, totals_pageviews=9, totals_transactions=0, label=0)

In [ ]:
train_spark = train_spark.drop('totals_totalTransactionRevenue').na.fill({'totals_bounces':0.0, 'totals_newVisits':0.0,
                                                                          'totals_timeOnSite':0.0, 'totals_pageviews':0.0,
                                                                          'totals_transactions':0.0})
X_train_spark, X_val_spark, X_test_spark = train_spark.randomSplit([0.8, 0.1, 0.1], seed=RANDOM_SEED)

In [ ]:
spark_lr = LogisticRegression(maxIter=100000, featuresCol='features',labelCol='label')
paramGrid = ParamGridBuilder()\
    .addGrid(spark_lr.regParam, [0.1, 0.01])\
    .build()

cols = [col for col in X_train_spark.columns if (col not in set(['totals_transactions','label','fullVisitorId']))]
vec = VectorAssembler(inputCols=cols, outputCol='features', handleInvalid='keep')

pipeline = Pipeline(stages=[vec, spark_lr])
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=10)

In [ ]:
crossval = crossval.fit(X_train_spark)
model1 = crossval.bestModel.stages[1]

In [ ]:
prediction = model1.predict(X_val_spark)
selected = prediction.select("probability", "prediction")